<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Opt_T7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Definimos las funciones para calcular el gradiente y Hessiano de la función de Rosembrok.

In [203]:
def Rosembrok(X):#X es un vector.
    return np.sum(100*(X[1:]-X[:-1]**2)**2+(1-X[:-1])**2)
def Rosembrok_grad(X):
    n=len(X)
    grad=np.zeros(n)
    grad[1:-1]=-400*X[1:-1]*(X[2:]-X[1:-1]**2)+200*(X[1:-1]-X[:-2]**2)-2*(1-X[1:-1])
    grad[0]=-400*X[0]*(X[1]-X[0]**2)-2*(1-X[0])
    grad[-1]=200*(X[-1]-X[-2]**2)
    return grad
def Rosembrok_Hessian(X):
    banda_s=X[:-1]
    H=np.diag(banda_s,1)
    H=-400*H+-400*H.T
    main_diag=-400*(X[2:]-X[1:-1]**2)+800*(X[1:-1]**2)+202
    np.fill_diagonal(H[1:-1,1:-1],main_diag)
    H[0,0]=(-400*(X[1]-X[0]**2)+800*(X[0]**2)+2)
    H[-1,-1]=200
    H[0,1]=-400*(X[1])
    H[-1,-2]=-400*X[-2]
    return(H)

In [19]:
Rosembrok(np.zeros(100))

99.0

Definimos las funciones para calcular el gradiente y Hessiano de la función de Wood.


In [104]:
def Wood(X):
    return (100*(X[0]**2-X[1])**2+(X[0]-1)**2+(X[2]-1)**2+90*(X[2]**2-X[3])**2+10.1*((X[1]-1)**2+(X[3]-1)**2)+19.8*(X[1]-1)*(X[3]-1))
def Wood_grad(X):
    G=[]
    G.append(400*X[0]*(X[0]**2-X[1])+2*(X[0]-1))
    G.append(-200*(X[0]**2-X[1])+20.2*(X[1]-1)+19.8*(X[3]-1))
    G.append(2*(X[2]-1)+360*(X[2]**2-X[3])*X[2])
    G.append(-180*(X[2]**2-X[3])+20.2*(X[3]-1)+19.8*(X[1]-1))
    return np.array(G)
def Wood_Hessian(X):
    D1=[2+400*(X[0]**2-1)+800*X[0]**2,-400*X[0],0,0]
    D2=[-400*X[0],220.2,0,19.8]
    D3=[0,0,2+720*(X[2]**2)+360*(X[2]-X[3]),-360*X[2]]
    D4=[0,19.8,-360*X[2],200.2]
    return np.array([D1,D2,D3,D4])

Definimos las funciones para calcular el gradiente y Hessiano de a función de Branin.


In [113]:
a=1
b=5.1/(4*np.pi**2)
c=5/np.pi
r=6
s=10
t=1/(8*np.pi)
def Branin(X):
    return (a*(X[1]-b*X[0]**2+c*X[0]-r)**2+s*(1-t)*np.cos(X[0])+s)
def Branin_grad(X):
    return np.array([2*a*(X[1]-b*X[0]**2+c*X[0]-r)*(-2*X[0]+c)-s*(1-t)*np.sin(X[0]),2*a*(X[1]-b*X[0]**2+c*X[0]-r)])
def Branin_Hessian(X):
    R1=np.array([2*a*(X[1]-b*X[0]**2+c*X[0]-r)*(-2*b)+(-2*b*X[0]+c)*(-4*a*b*X[0]+2*a*c)-s*(1-t)*np.cos(X[0]),2*a*(-2*b*X[0]+c)])
    R2=np.array([2*a*(-2*b*X[0]+c),2*a])
    return np.array([R1,R2])

A continuncación implementamos el método de región de confianza usando la modificación de Dog-leg.

In [163]:
def conseguir_p_k():
    pass
def get_Newton(grad,B):
    return np.linalg.solve(B,-grad)
def get_Cauchy(grad,B):
    return (grad*(-(np.dot(grad,grad))/(np.dot(grad,np.dot(B,grad)))))
def get_Newton_intersection(a,b,c):#Formula para hallar la interseccion
    return ((-b+np.sqrt(b**2-4*a*c))/(2*a))
def get_Cauchy_boundary(vector,radio):#Regresa un punto en la direccion del vector, de radio dado.
    return (radio/np.linalg.norm(vector))*vector
def update_radio(radio_k,radio_global,rho,p_k):
    if rho<0.25:
        return 0.25*radio_k
    else:
        if rho>0.75:
            return np.min([2*radio_k,radio_global]) 
        else:
             return radio_k
    

In [188]:
def Dog_leg(x_0,f,grad_f,Hessian_f,radio_global,etha_1=0.25,tol_g=10**(-4),max_iter=10**3):
    x_k=x_0
    radio_k=radio_global
    progr=[]
    for k in range(max_iter):
        grad_k=grad_f(x_k)
        f_k=f(x_k)
        H_k=Hessian_f(x_k)
        if(np.linalg.norm(grad_k)<tol_g):
            break
        #Conseguimos $p_k$
        p_k_B=get_Newton(grad_k,H_k)#Newton sin restricciones
        if (np.linalg.norm(p_k_B)>=radio_k):#Fuera de la region
            p_k_U=get_Cauchy(grad_k,H_k)
            if(np.linalg.norm(p_k_U)>=radio_k): #Fuera de la region
                #Buscamos la interseccion
                p_k= get_Cauchy_boundary(p_k_U,radio_k)
            else:# p_k_U Dentro de la region 
                #Buscamos interseccion de Newton
                lamb=get_Newton_intersection(a=np.linalg.norm(p_k_B-p_k_U)**2,b=2*np.dot(p_k_U,p_k_B-p_k_U),c=np.linalg.norm(p_k_U)**2-radio_k**2)
                p_k=p_k_U+lamb*(p_k_B-p_k_U)

        else:
            p_k=p_k_B
            print("Caso Newton_no inter")
        #Calcular medida de ajuste
        rho=(f_k-f(x_k+p_k))/(-(np.dot(grad_k,p_k)+0.5*np.dot(p_k,np.dot(H_k,p_k))))
        if(rho>etha_1):
            #Actualizamos x_k
            x_k_next=x_k+p_k
            x_k=x_k_next
            print("f",f(x_k))
        else:
            print("Mal paso")
        radio_k=update_radio(radio_k,radio_global,rho,p_k)
        progr.append(f(x_k))
    return 0



Implementamos el algoritmo de Newton-Cauchy alterno

In [194]:
def Cauchy_Newton(x_0,f,grad_f,Hessian_f,radio_global,etha_1=0.25,tol_g=10**(-4),max_iter=10**3):
    x_k=x_0
    radio_k=radio_global
    progr=[]
    for k in range(max_iter):
        grad_k=grad_f(x_k)
        f_k=f(x_k)
        H_k=Hessian_f(x_k)
        if(np.linalg.norm(grad_k)<tol_g):
            break
        #Conseguimos $p_k$
        p_k_B=get_Newton(grad_k,H_k)#Newton sin restricciones
        if (np.linalg.norm(p_k_B)>=radio_k):#Fuera de la region
            p_k=get_Cauchy(grad_k,H_k)
        else:
            p_k=p_k_B
        #Calcular medida de ajuste
        rho=(f_k-f(x_k+p_k))/(-(np.dot(grad_k,p_k)+0.5*np.dot(p_k,np.dot(H_k,p_k))))
        if(rho>etha_1):
            #Actualizamos x_k
            x_k_next=x_k+p_k
            x_k=x_k_next
        radio_k=update_radio(radio_k,radio_global,rho,p_k)
        progr.append(f(x_k))
    return x_k

In [ ]:
np.random.seed(10)
x_0=np.random.normal(0,0.5,size=100)+1
Dog_leg(x_0=x_0,f=Rosembrok,grad_f=Rosembrok_grad,Hessian_f=Rosembrok_Hessian,radio_global=0.5,etha_1=0.2,tol_g=10**(-4),max_iter=500)

In [ ]:
np.random.seed(10)
x_0=np.random.normal(0,0.5,size=100)+1
Cauchy_Newton(x_0=x_0,f=Rosembrok,grad_f=Rosembrok_grad,Hessian_f=Rosembrok_Hessian,radio_global=0.5,etha_1=0.2,tol_g=10**(-4),max_iter=500)

In [ ]:
np.random.seed(15)
x_0=np.random.normal(0,0.5,size=4)+1
Dog_leg(x_0=x_0,f=Wood,grad_f=Wood_grad,Hessian_f=Wood_Hessian,radio_global=0.25,etha_1=0.25,tol_g=10**(-4),max_iter=50)

In [ ]:
np.random.seed(15)
x_0=np.random.normal(0,0.5,size=2)+1
Dog_leg(x_0=x_0,f=Branin,grad_f=Branin_grad,Hessian_f=Branin_Hessian,radio_global=0.55,etha_1=0.25,tol_g=10**(-4),max_iter=150)

In [191]:
import numpy as np
from numba import njit
import time
from datetime import timedelta

In [210]:
Funcion_name=['Rosembrok','Wood','Branin']
algo_name=['Cauchy-Newton Alterno', 'Newton modificado',"Dog-leg"]
functions=[Rosembrok,Wood,Branin]
gradients=[Rosembrok_grad,Wood_grad,Branin_grad]
Hessians=[Rosembrok_Hessian,Wood_Hessian,Branin_Hessian]
for w in range(3):
        print('Tiempo promedio para '+Funcion_name[w]+' usando ' +algo_name[0] +'\n')
        tiempos=[]
        for i in range(3):
            start = time.monotonic()
            np.random.seed(i)
            if(w==0):
                x_0=np.random.uniform(-2,2,size=100)
            if(w==1):
                x_0=np.random.uniform(-2,2,size=4)
            if(w==2):
                x_0=np.random.uniform(-2,2,size=2)
            C1 = Cauchy_Newton(x_0=x_0,f=functions[w],grad_f=gradients[w],Hessian_f=Hessians[w],radio_global=0.5,etha_1=0.2,tol_g=10**(-4),max_iter=10**4)
            end = time.monotonic()
            tiempos.append(timedelta(seconds=end - start).total_seconds())
        print(np.mean(tiempos))
        print('-----------------------')
        print('Tiempo promedio para '+Funcion_name[w]+' usando ' +algo_name[1] +'\n')
        tiempos=[]
        for i in range(3):
            start = time.monotonic()
            np.random.seed(i)
            if(w==0):
                x_0=np.random.uniform(-2,2,size=100)
            if(w==1):
                x_0=np.random.uniform(-2,2,size=4)
            if(w==2):
                x_0=np.random.uniform(-2,2,size=2)
            C1 = Newton_mod(x_0=x_0,f=functions[w],grad_f=gradients[w],Hessian_f=Hessians[w],radio_global=0.5,etha_1=0.2,tol_g=10**(-4),max_iter=10**4)
            end = time.monotonic()
            tiempos.append(timedelta(seconds=end - start).total_seconds())
        print(np.mean(tiempos))
        print('-----------------------')
        print('Tiempo promedio para '+Funcion_name[w]+' usando ' +algo_name[2] +'\n')
        tiempos=[]
        for i in range(3):
            start = time.monotonic()
            np.random.seed(i)
            if(w==0):
                x_0=np.random.uniform(-2,2,size=100)
            if(w==1):
                x_0=np.random.uniform(-2,2,size=4)
            if(w==2):
                x_0=np.random.uniform(-2,2,size=2)
            C1 = Dog_leg(x_0=x_0,f=functions[w],grad_f=gradients[w],Hessian_f=Hessians[w],radio_global=0.5,etha_1=0.2,tol_g=10**(-4),max_iter=10**4)
            end = time.monotonic()
            tiempos.append(timedelta(seconds=end - start).total_seconds())
        print(np.mean(tiempos))
        print('-----------------------')


Tiempo promedio para Rosembrok usando Cauchy-Newton Alterno



KeyboardInterrupt: ignored